# Purpose:

2015-02-12 (Thursday)

use pandas to get Gisella something more like what she said she wanted in terms of summary tables for the _G. pallidipes_ summary table information:

- data types:
    - location
    - symbols when present (_I assume this mean location symbol?_)
    - number of individuals
    - date range
    - is tissue?
    - is extraction?
    - analysis status

__Notes:__

- group by location
- filter on Kenya locations (not sure which locations are kenya if not explicit)




# Implementation:

## Imports:

In [1]:
# imports
import pandas as pd
import numpy as np

import qgrid as qg

In [2]:
qg.nbinstall()

In [3]:
qshow = qg.show_grid

## File paths:

In [4]:
# define paths to files

xl_path = '/home/gus/Documents/YalePostDoc/project_stuff/G_pallidipes_kenya/Collection_data_updated_Feb_9_2015.xlsx'

## Load data

In [5]:
xl = pd.ExcelFile(xl_path)

In [6]:
xl.sheet_names

[u'cold room',
 u'idaho boxes',
 u'Idaho strip tube boxes',
 u'gisella freezer',
 u'-80 freezer',
 u'dissection_sheet_template']

In [7]:
sample_cat = xl.parse(sheetname=u'dissection_sheet_template', 
         header=0, 
         skiprows=None, skip_footer=0, 
         index_col=None, parse_cols=None, 
         parse_dates=False, date_parser=None, 
         na_values=['NA'], 
         thousands=None, chunksize=None, 
         convert_float=False, 
         has_index_names=False, converters=None)

In [8]:
qshow(sample_cat,remote_js=True)

## Manipulate table to facilitate pivot operations

In [9]:
# add a number column to represent that each row is ONE sample
sample_cat['Count'] = 1

### Convert Gpd to Gp

In [10]:
try:
    gpd_mask.head()
except NameError:
    gpd_mask = sample_cat.Species == 'Gpd'


In [11]:
sample_cat.loc[gpd_mask,'Species'] = 'Gp'

In [12]:
sample_cat[gpd_mask].head(1)

Lab_Source               Village Trap_No  Date  \
2135  EPH Idaho Samples (Tube Boxes)  Galana (?) D1, Kenya       0  Unk.   

     Species Sex  Teneral  Dead Fly_Number  Hunger_stage  ...   midgut  \
2135      Gp   M      NaN   NaN        018           NaN  ...      NaN   

      sal_gland  Kept_in                                            Comment  \
2135        NaN     Unk.  CTAB, color coded aliquots, poorly labelled an...   

     Tube_or_box Tissue Derivative Method_of_prep Box_number_id Count  
2135         box    NaN       gDNA            NaN            50     1  

[1 rows x 22 columns]

### Convert Date: Unk, Unk., etc to single value (Unk)

In [13]:
def date_unk_func(x):
    try:
        if x.upper().startswith('UNK'):
            return 'UNK'
        else:
            return x
    except AttributeError:
        return x
    
def standardize_date_unk(df):
    new = df.Date.apply(date_unk_func)
    df.Date = new

In [14]:
standardize_date_unk(sample_cat)

### Add tissue_or_derivative column

In [15]:
sample_cat['tissue_or_extraction'] = 'none'

In [16]:
# generate masks of which rows match tissue or derivative
try:
    is_tissue_mask.head()
except NameError:
    is_tissue_mask =  sample_cat.Tissue.apply(lambda x: isinstance(x, (unicode, str)))
    
try:
    is_derivative_mask.head()
except NameError:
    is_derivative_mask =  sample_cat.Derivative.apply(lambda x: isinstance(x, (unicode, str)))

In [17]:
sample_cat.loc[is_tissue_mask, 'tissue_or_extraction'] = 'tissue'
sample_cat.loc[is_derivative_mask, 'tissue_or_extraction'] = 'extraction'

## Begin Pivots

In [18]:
ptab = pd.pivot_table(sample_cat,index=['Village', 'Lab_Source','Date', 'Species'],
                      values=['Count'],
                      columns=['tissue_or_extraction'],
                      fill_value=0,
                      aggfunc=[np.sum])

In [19]:
ptab.query('Species == ["Gp"]')

sum  \
                                                                                                                Count   
tissue_or_extraction                                                                                       extraction   
Village                                   Lab_Source                           Date                Species              
"Africa" P.M.S.                           EPH Idaho Samples (Tube Boxes)       2002-11/12          Gp              46   
Arba Minch, Ethiopia                      EPH -80                              1997-01-11 00:00:00 Gp              18   
                                          EPH Idaho Samples (Strip Tube Boxes) 1997-01             Gp              24   
                                                                               UNK                 Gp              48   
                                          EPH Idaho Samples (Tube Boxes)       UNK                 Gp              32   
Arba Minch, Ethiopia, P3                  EPH Idaho Samples (Tube Boxes)       UNK                 Gp              24   
Arba Minch, Ethiopia, P6                  EPH Idaho Samples (Tube Boxes)       UNK                 Gp              24   
Block A (Ruma?), Kenya                    EPH Idaho Samples (Strip Tube Boxes) 2003-11-02          Gp             400   
Busia, Kenya                              EPH Idaho Samples (Strip Tube Boxes) UNK                 Gp              48   
Campsite                                  EPH Idaho Samples (Strip Tube Boxes) 2003-12-12          Gp              34   
Changara, Mozambique                      EPH Idaho Samples (Strip Tube Boxes) UNK                 Gp              54   
Dakabuko, Galana, Kenya                   EPH Idaho Samples (Strip Tube Boxes) 2000-02-25          Gp              96   
                                                                               UNK                 Gp              24   
Deme River, Ethiopia                      EPH Idaho Samples (Strip Tube Boxes) UNK                 Gp              24   
Galana (?)  K5, Kenya                     EPH Idaho Samples (Tube Boxes)       UNK                 Gp               9   
Galana (?) D1, Kenya                      EPH Idaho Samples (Tube Boxes)       UNK                 Gp               8   
Galana (?) K2, Kenya                      EPH Idaho Samples (Tube Boxes)       UNK                 Gp               7   
Galana, Kenya                             Gisella Freezer Samples              2000-02             Gp              24   
Kabukanga, Uganda                         Gisella Freezer Samples              2008-04             Gp              40   
                                                                               UNK                 Gp               4   
Kabunkanga, Kibale District, Kenya        Cold Room Samples (156b)             2010-09-23          Gp               0   
                                                                               2010-09-24          Gp               0   
                                                                               2010-09-25          Gp               0   
Kakumbi, Zambia                           EPH Idaho Samples (Strip Tube Boxes) UNK                 Gp              24   
Kalema, Kenya                             EPH Idaho Samples (Strip Tube Boxes) 2000                Gp              18   
Kapesur, Kenya                            Cold Room Samples (156b)             2009-03             Gp               0   
                                                                               2009-07             Gp               0   
                                          Gisella Freezer Samples              2009-03             Gp               6   
                                                                               2009-03-15          Gp              96   
Kathekani, Kenya                          EPH Idaho Samples (Strip Tube Boxes) 2000-12-22          Gp              56   
